In [1]:
import os
import pandas as pd
import h3
import random
from tqdm import tqdm
from collections import defaultdict
import pickle
import psutil
import time

In [2]:
file_path = '/disk_4/uddeshya/Dataset/combined_cleaned_triplet_train_PUNJAB_05.csv'

triplet_df = pd.read_csv(file_path)

In [3]:
triplet_df.head()

city  pincode                                    cleaned_address  \
0  Moga District   142048                      00 baghela wala Moga District   
1           Moga   142048               02 isf college of pharmacy moga Moga   
2           Moga   142048           1 dhalle_ke br canda zira road moga Moga   
3  Moga District   142048   1 goyal rice mills jira road khosa pando Moga...   
4           Moga   142048                                 1 mothan wali Moga   

   lookup_lat  lookup_lng  no_of_delivereies  
0   30.807182   74.997443                  6  
1   30.827942   75.118181                  6  
2   30.850509   75.137326                  7  
3   30.850573   75.134123                  9  
4   30.782533   75.098455                 10

In [4]:
triplet_df['cleaned_address'][1]

' 02 isf college of pharmacy moga Moga'

In [5]:
def generate_triplets_from_csv(csv_path, resolution, K, T, D, output_pickle_path):
    df = pd.read_csv(csv_path)
    addresses = df['cleaned_address'].tolist()
    lat_lngs = df[['lookup_lat', 'lookup_lng']].values.tolist()

    def generate_h3_cells(lat_lngs, resolution):
        return [h3.geo_to_h3(lat, lng, resolution) for lat, lng in lat_lngs]

    h3_cells = generate_h3_cells(lat_lngs, resolution)

    def is_valid_h3_cell(cell):
        try:
            h3.h3_get_resolution(cell)
            return True
        except h3.H3CellError:
            return False

    def generate_triplets_on_the_fly(h3_cells, K, T, D, resolution):
        cell_to_points = defaultdict(list)
        h3_cells_set = set(h3_cells)

        for index, cell in enumerate(h3_cells):
            if is_valid_h3_cell(cell):
                cell_to_points[cell].append(index)

        triplets = []
        for cell, points in tqdm(cell_to_points.items(), desc="Generating Triplets", unit="cell"):
            if len(points) > 1:
                current_resolution = h3.h3_get_resolution(cell)
                if current_resolution >= resolution:
                    try:
                        parent_cell = h3.h3_to_parent(cell, resolution - 1)
                    except (h3.H3CellError, h3.H3ResolutionError):
                        continue

                    rings = []
                    for ring_level in range(1, D + 1):
                        if ring_level == 1:
                            ring = h3.k_ring(parent_cell, 2) - h3.k_ring(parent_cell, 1)
                        else:
                            previous_ring = h3.k_ring(parent_cell, ring_level)
                            ring = h3.k_ring(parent_cell, ring_level + 1) - previous_ring
                        rings.append(ring)

                valid_negative_samples_by_level = {ring_level: set() for ring_level in range(1, D + 1)}
                for ring_level, ring in enumerate(rings, start=1):
                    for neighbor in ring:
                        children = h3.h3_to_children(neighbor, resolution)
                        random.shuffle(list(children))
                        for child in children:
                            if child in h3_cells_set:
                                valid_negative_samples_by_level[ring_level].add(child)
                                if len(valid_negative_samples_by_level[ring_level]) >= 10000:
                                    break
                        if len(valid_negative_samples_by_level[ring_level]) >= 10000:
                            break

                for point in points:
                    positive_points = [p for p in points if p != point]
                    if positive_points:
                        sampled_positives = random.sample(positive_points, min(K, len(positive_points)))
                        for positive_sample in sampled_positives:
                            for ring_level in range(1, D + 1):
                                valid_negative_samples = valid_negative_samples_by_level[ring_level]
                                if valid_negative_samples:
                                    sampled_negatives = random.sample(list(valid_negative_samples), min(T, len(valid_negative_samples)))
                                    for neg in sampled_negatives:
                                        neg_index = h3_cells.index(neg)
                                        triplets.append((point, positive_sample, neg_index, ring_level))

        return triplets

    triplets = generate_triplets_on_the_fly(h3_cells, K, T, D, resolution)

    with open(output_pickle_path, 'wb') as f:
        pickle.dump({
            'triplets': triplets,
            'addresses': addresses,
            'lat_lngs': lat_lngs
        }, f)

    print(f"Triplets saved to {output_pickle_path}")

In [6]:
csv_path = '/disk_4/uddeshya/Dataset/combined_cleaned_triplet_train_PUNJAB_05.csv'
resolution = 10
K = 1  
T = 1 
D = 3  
output_pickle_path = '/disk_4/uddeshya/Dataset/combined_triplets_PUNJAB_05.1.pkl'

In [ ]:
generate_triplets_from_csv(csv_path, resolution, K, T, D, output_pickle_path)

Generating Triplets:   1%|          | 680/103815 [01:43<1:07:32, 25.45cell/s] 